In [1]:
# Vikram & Sahana
# Jan 28 2022
# Classical Machine Learning Model V4
# Detecting Severity of Alzheimer's with brain scans more accurately

# Differences from V3
#   Quantum Enhanced

In [2]:
from tqdm import tqdm
import os
import cv2
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from random import shuffle
import matplotlib.pyplot as plt
import tflearn
import nibabel as nib
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, image_preloader
from tflearn.metrics import Accuracy
import numpy as np
import pandas as pd
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
import numpy as np
from glob import glob
from skimage.io import imread
import tensorflow as tf
tf.get_logger().setLevel('INFO')
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from time import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer, IBMQ
from qiskit.compiler import transpile
from time import perf_counter

ModuleNotFoundError: ignored

In [ ]:
def process_images(images_set, image_info):
    print()
    print("Processing Images...")
    df = pd.read_csv(CSV_DIR) # reads in csv file
    df["Downloaded"] = "" # creates new Downloaded column


    for index, row in df.iterrows(): # goes through every row

        # CONNECTING EACH IMAGE TO ROW IN THE
        id = row['Image ID']
        for i in range(len(images_set)): # checks through all the images
            if "I%s" % id in image_info[i][0]:
                # if the image id is in the file name of the image being checked
                df.at[index,'Downloaded'] = i # store the index
                image_info[i][1] = True

        if (row['Research Group'] in RG_CLASSIFICATIONS.keys()):
            df.at[index,'Research Group'] = RG_CLASSIFICATIONS[row['Research Group']]
        else:
            df.at[index,'Research Group'] = -1
        # CHANGING ANY COL,ROW WHERE IT SAYS NAN
        for col in df:
            if index > 0 and pd.isnull(row[col]):
                if row["Subject ID"] == df.at[index-1,"Subject ID"]:
                    df.at[index, col] = df.at[index-1,col]

        for colandval in INCLUDE_DATA_ROWS:
            coltoremove = colandval[0]
            valtoremove = colandval[1]
            if (valtoremove not in df.at[index, coltoremove]):
                df.at[index,'Research Group'] = -1

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)


    df2 = df.loc[df['Downloaded'] != ""]
    df3 = df2.loc[df2['Research Group'] != -1]
    print("Processed!")
    return df3

def process_lattributes(df, dat, verbose=True):
    print()
    if verbose: print("Processing Scan Attributes...")
    cs = MinMaxScaler()
    continuous = cs.fit_transform(dat[INCLUDE_DATA_NUMBERS])
    categoricals = []
    for i in INCLUDE_DATA_CATEGORIES:
        zipBinarizer = LabelBinarizer().fit(df[i])
        categorical = zipBinarizer.transform(dat[i])
        categoricals.append(categorical)

    x = np.hstack([continuous, *categoricals])
    if verbose: print("Processed!")
    return x

def get_data():
    print()
    print("Getting Images...")
    path = IMAGE_DIR
    images = []
    image_info = []
    for fol1 in tqdm(os.listdir(path), disable=True):
        if fol1 == ".DS_Store": continue
        path1 = os.path.join(path, fol1)
        for fol2 in tqdm(os.listdir(path1), disable=True):
            if fol2 == ".DS_Store": continue
            path2 = os.path.join(path1, fol2)
            for fol3 in tqdm(os.listdir(path2), disable=True):
                if fol3 == ".DS_Store": continue
                path3 = os.path.join(path2, fol3)
                for fol4 in tqdm(os.listdir(path3), disable=True):
                    if fol4 == ".DS_Store": continue
                    path4 = os.path.join(path3, fol4)
                    # After going through all of the subfolders, we can finally see the image
                    for img in tqdm(os.listdir(path4), disable=True):
                        if img == ".DS_Store": continue
                        # print(img)
                        try:
                            path5 = os.path.join(path4, img)
                            image_data = nib.load(path5).get_fdata() # get the numbers of the 3D image
                            image_data = np.rot90(image_data) # make it axial
                            """
                            our_slice = image_data[image_data.shape[0]//2] # get the very middle slice
                            our_slice_reshaped = scale_array(our_slice, IMG_SHAPE)
                            images.append(our_slice_reshaped)
                            plt.imshow(our_slice_reshaped)
                            """

                            big_image = np.empty([IMG_SHAPE[0], IMG_SHAPE[1]*NUM_SLICES, 1])
                            for i in range(NUM_SLICES):
                                our_slice = image_data[image_data.shape[0]*(i+1)//(NUM_SLICES+1)]
                                our_slice_reshaped = scale_array(our_slice, IMG_SHAPE)
                                for r in range(IMG_SHAPE[0]):
                                    for c in range(IMG_SHAPE[1]):
                                        this_number = our_slice_reshaped[r,c,0]
                                        big_image[r,c+(IMG_SHAPE[1]*i),0] = this_number

                            # print(big_image.shape)
                            # plt.imshow(big_image)
                            images.append(big_image) # add it to the list


                            image_info.append([img, False])
                        except:
                            pass
    print("Got!")
    return images, image_info

def get_classical_images(override = False, save = True):
    print()
    start = int(round(time()*1000))
    df_save_name = SAVE_PATH + "/data_" + DATA_SAVE_FILENAME + ".csv"
    image_save_name = SAVE_PATH + "/c_images_" + DATA_SAVE_FILENAME + ".npy"
    print("Getting Images and Data...")
    try:
        c_images = np.load(image_save_name)
        df = pd.read_csv(df_save_name)
    except:
        override = True

    if override:
        images_set, image_info = get_data()
        print("There are %s images" % len(images_set))
        # view_image()


        df = process_images(images_set, image_info)
        print("DF's Shape: %s" % str(df.shape))

        print()
        print("Classical Images preprocessing...")
        c_images = []
        for index in list(df["Downloaded"]):
            c_images.append(images_set[index])
        c_images = np.array(c_images)

        print("Saving images to %s" % image_save_name)
        print("Saving dataframe to %s" % df_save_name)
        if save: df.to_csv(df_save_name, index=False)
        if save: np.save(image_save_name, c_images)
        print("Preprocessed!")

    override = False
    c_images = np.load(image_save_name)
    df = pd.read_csv(df_save_name)
    end = int(round(time()*1000))
    pro_time = (end-start)/1000
    print()
    print("Got!")
    print("That took %s minutes" % (pro_time / 60))
    return df, c_images

In [ ]:
def create_mlp(dim, regress=False, verbose=True):
    if (verbose): print("Creating MLP Classifier for attributes...")
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))
    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))
    # return our model
    if (verbose): print("Created!")
    return model

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False, verbose=True):
    if (verbose): print("Creating CNN for images...")
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (width, height, depth)
    chanDim = -1
    # define the model input
    inputs = Input(shape=inputShape)
    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs
        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)
    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)
    # construct the CNN
    model = Model(inputs, x)
    # return the CNN
    if (verbose): print("Created!")
    return model

In [ ]:
def scale_array(x, new_size):
    new_size = (new_size[0]/x.shape[0],new_size[1]/x.shape[1], 1)
    y = scipy.ndimage.zoom(x, new_size)
    return y

def score(model, testNumX, testImagesX, testY, verbose=False):
    score = 0
    start = int(round(time()*1000))
    pred = model.predict([testNumX, testImagesX])
    end = int(round(time()*1000))
    for i in range(len(pred)):
        prediction = np.where(pred[i] == np.amax(pred[i]))[0][0]
        real = np.where(testY[i] == 1)[0][0]
        # print(pred[i], testY[i])
        if verbose: print("Predicted Class vs. Actual Class: %s %s" % (prediction, real))
        if (prediction == real):
            score+=1
        else:
            if verbose: print("   Incorrect!")
    return score / len(pred), end-start

In [ ]:
dev = qml.device("default.qubit", wires=4)
@qml.qnode(dev)
def circuit(phi):


    # Random circuit parameters
    rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, 4))
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(4)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    rr = IMG_SHAPE[0]
    cc = IMG_SHAPE[1]*NUM_SLICES
    out = np.zeros((rr//2, cc//2, 4))

    # Loop over the coordinates of the top-left pixel of 2X2 squares
    for j in range(0, rr, 2):
        for k in range(0, cc, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

def preprocess_quantum(images, override=False, save = True):
    start = int(round(time()*1000))
    print("Getting Quantum Images...")
    image_save_path = SAVE_PATH + "/q_images" + DATA_SAVE_FILENAME + '.npy'
    try:
        np.load(image_save_path)
    except:
        override = True
    if override == True:
        q_images = []
        print("Preprocess images...\n")
        for idx, img in enumerate(images):
            print("{}/{}        ".format(idx + 1, len(images)), end="\r")
            q_images.append(quanv(img))
        q_images = np.asarray(q_images)
        print("Preprocessed!")

        # Save pre-processed images


        print("Saving to %s" % image_save_path)
        if save: np.save(image_save_path, q_images)


    # Load pre-processed images
    override = False
    q_images = np.load(image_save_path)
    end = int(round(time()*1000))
    preprocess_time = (end-start)/1000
    print("\nGot!")
    print("That took %s mins - (%s sec)" % (round(preprocess_time/60, 2), round(preprocess_time, 2)))
    return q_images

In [ ]:
# OPTIONAL

def view_image():
    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import skimage, os
    from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
    from skimage.measure import label,regionprops, perimeter
    from skimage.morphology import binary_dilation, binary_opening
    from skimage.filters import roberts, sobel
    from skimage import measure, feature
    from skimage.segmentation import clear_border
    from skimage import data
    from scipy import ndimage as ndi
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    import scipy.misc
    import numpy as np
    from glob import glob
    from skimage.io import imread
    BASE_IMG_PATH = "/Users/sidharthanantha/Documents/SciFair 2022/ADNI/002_S_6030/Axial_T2_STAR/2017-06-15_13_30_22.0/S573170/ADNI_002_S_6030_MR_Axial_T2_STAR__br_raw_20170616113815278_41_S573170_I861966.nii"

    image = glob(os.path.join(BASE_IMG_PATH, '*'))
    image = glob(BASE_IMG_PATH)[0]
    print(image)
    %matplotlib inline
    try:
        import nibabel as nib
    except:
        raise ImportError('bip bop')

    image_data = nib.load(image).get_data()
    image_data = np.rot90(image_data)
    # image_data = np.reshape(image_data, (44, 256, 256, 1))
    # image_data = np.rot90(image_data) # make it axial
    # print(image_data.shape)
    le_img = image_data[image_data.shape[0]//2]
    plt.imshow(le_img)
    # print(le_img[len(le_img)//2])
    # print(le_img.shape)
    le_img = scale_array(le_img, (70,70))
    plt.imshow(le_img)
    # print(image_data[2])


def show_q_images(c_images, q_images):
    n_samples = 4
    n_channels = 4
    fig, axes = plt.subplots(1 + n_channels, n_samples, figsize=(10, 10))
    for k in range(n_samples):
        axes[0, 0].set_ylabel("Input")
        if k != 0:
            axes[0, k].yaxis.set_visible(False)
        axes[0, k].imshow(c_images[k, :, :, 0], cmap="gray")

        # Plot all output channels
        for c in range(n_channels):
            axes[c + 1, 0].set_ylabel("Output [ch. {}]".format(c))
            if k != 0:
                axes[c, k].yaxis.set_visible(False)
            axes[c + 1, k].imshow(q_images[k, :, :, c], cmap="gray")

    plt.tight_layout()
    plt.show()

In [ ]:
n_layers = 1

EPOCHS = 200
LR = 1e-3

np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
VERSION = 'V4_FINAL' # used for saving model name
DATA_VERSION = 'V3' # used for retrieving imgs + csv, and saving imgs + df

SAVE_PATH = "/Users/sidharthanantha/Documents/SciFair 2022/SaveData" # path for saving imgs, models, df
IMAGE_DIR = "/Users/sidharthanantha/Documents/SciFair 2022/Alzheimers_Dataset_%s" % (DATA_VERSION)
CSV_DIR = "/Users/sidharthanantha/Documents/SciFair 2022/classical_v2_attr_data.csv"

IMG_SHAPE = (70, 70)
MODEL_NAME = "classifying_alzheimers_%s.model" % VERSION # saves ml model under this name
Y_ATTR = "Research Group"

INCLUDE_DATA_NUMBERS = ['Weight','APOE A1','APOE A2','Age','MMSE Total Score','GDSCALE Total Score','Global CDR','FAQ Total Score','NPI-Q Total Score']
INCLUDE_DATA_CATEGORIES = ['Sex', 'Description','Type','Imaging Protocol','Structure']
INCLUDE_DATA_ROWS = [
    ['Imaging Protocol', 'Field Strength=3.0'],
#     ['Imaging Protocol', 'Acquisition Type=2D'],
#     ['Imaging Protocol', 'Weighting=PD'],
]

RG_CLASSIFICATIONS = {
    "CN": 0,
    "AD": 1,
    "MCI": 2,
    "EMCI": 2,
    "LMCI": 2,
    "SMC": 2,
}
NUM_SLICES = 1 # how many brain slices
NUM_TIMES_TEST = 5 # how many times should the alg be tested
NUM_CLASSES = len(list(set(RG_CLASSIFICATIONS.values())))

DATA_SAVE_FILENAME = "%s_%sslices_%sclasses_%smodel" % (DATA_VERSION, NUM_SLICES, NUM_CLASSES, VERSION) # to save dfs and imgs

In [ ]:
df, c_images = get_classical_images()
print("Shape of the Image array: %s\n" % str(np.shape(c_images)))


Getting Images and Data...

Getting Images...
Got!
There are 11855 images

Processing Images...
Processed!
DF's Shape: (1537, 22)

Classical Images preprocessing...
Saving images to /Users/sidharthanantha/Documents/SciFair 2022/SaveData/c_images_V3_1slices_3classes_V4_FINALmodel.npy
Saving dataframe to /Users/sidharthanantha/Documents/SciFair 2022/SaveData/data_V3_1slices_3classes_V4_FINALmodel.csv
Preprocessed!

Got!
That took 13.079716666666666 minutes
Shape of the Image array: (1537, 70, 70, 1)



In [ ]:
df

,Subject ID,Sex,Weight,Research Group,APOE A1,APOE A2,Visit,Study Date,Age,Global CDR,...,GDSCALE Total Score,FAQ Total Score,Modality,Description,Type,Imaging Protocol,Image ID,Structure,Registration,Downloaded
0,002_S_0413,F,60.4,0,3.0,3.0,ADNI Baseline,5/19/2006,76.4,0.0,...,0.0,0.0,MRI,Double_TSE,Original,Slice Thickness=3.0;Matrix Z=52.0;Manufacturer...,15804,Brain,native,10459
1,002_S_0413,F,60.8,0,3.0,3.0,ADNI1/GO Month 6,11/15/2006,76.9,0.0,...,0.0,0.0,MRI,SURVEY,Original,Slice Thickness=10.0;Matrix Z=3.0;Manufacturer...,30120,Brain,native,8601
2,002_S_0413,F,60.1,0,3.0,3.0,ADNI1/GO Month 12,6/01/2007,77.5,0.0,...,1.0,0.0,MRI,Double_TSE,Original,Slice Thickness=3.0;Matrix Z=48.0;Manufacturer...,55787,Brain,native,3684
3,002_S_0413,F,60.1,0,3.0,3.0,ADNI1/GO Month 12,6/01/2007,77.5,0.0,...,1.0,0.0,MRI,SURVEY,Original,Slice Thickness=10.0;Matrix Z=3.0;Manufacturer...,55784,Brain,native,3681
4,002_S_0413,F,57.7,0,3.0,3.0,ADNI1/GO Month 24,7/31/2008,78.6,0.0,...,0.0,0.0,MRI,Double_TSE,Original,Slice Thickness=3.0;Matrix Z=51.0;Manufacturer...,115004,Brain,native,3683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1532,941_S_4377,F,123.4,2,3.0,4.0,ADNI2 Month 6-New Pt,8/16/2012,70.1,0.5,...,2.0,2.0,MRI,localizer,Original,Slice Thickness=8.0;Matrix Z=1.0;Acquisition T...,324374,Brain,native,10328
1533,941_S_4377,F,123.4,2,3.0,4.0,ADNI2 Month 6-New Pt,8/16/2012,70.1,0.5,...,2.0,2.0,MRI,localizer,Original,Slice Thickness=8.0;Matrix Z=1.0;Acquisition T...,324372,Brain,native,10329
1534,941_S_4377,F,123.4,2,3.0,4.0,ADNI2 Month 6-New Pt,8/16/2012,70.1,0.5,...,2.0,2.0,MRI,AXIAL_T2_STAR,Original,Slice Thickness=4.0;Matrix Z=44.0;Acquisition ...,324377,Brain,native,10327
1535,941_S_4420,M,75.3,2,3.0,3.0,ADNI2 Month 6-New Pt,12/21/2012,82.2,0.5,...,4.0,1.0,MRI,ADNI_gre_field_mapping,Original,Slice Thickness=3.0;Matrix Z=45.0;Acquisition ...,352803,Brain,native,363


In [ ]:
q_images = preprocess_quantum(c_images)
# show_q_images(c_images, q_images)

Getting Quantum Images...
Preprocess images...

Preprocessed!    
Saving to /Users/sidharthanantha/Documents/SciFair 2022/SaveData/q_imagesV3_1slices_3classes_V4_FINALmodel.npy

Got!
That took 313.42 mins - (18805.5 sec)


In [ ]:
def splite(split, verbose=True):
    (trainAttrX, testAttrX, trainImagesX, testImagesX) = split

    trainY = to_categorical(np.array(trainAttrX[Y_ATTR]))
    testY = to_categorical(np.array(testAttrX[Y_ATTR]))
    trainAttrX

    trainNumX = process_lattributes(df, trainAttrX, verbose=verbose)
    testNumX = process_lattributes(df, testAttrX, verbose=verbose)

    return trainNumX, testNumX, trainImagesX, testImagesX, trainY, testY

def make(trainNumX, testNumX, trainImagesX, testImagesX, trainY, testY, verbose=True):
    mlp = create_mlp(trainNumX.shape[1], regress=False, verbose=verbose)
    cnn = create_cnn(*np.shape(trainImagesX[0]), regress=False, verbose=verbose)
    combinedInput = concatenate([mlp.output, cnn.output])

    x = Dense(4, activation="relu")(combinedInput)
    x = Dense(NUM_CLASSES, activation="softmax")(x)
    model = Model(inputs=[mlp.input, cnn.input], outputs=x)
    opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
    model.compile(loss="categorical_crossentropy", optimizer=opt)
    return model

def traine(trainNumX, testNumX, trainImagesX, testImagesX, trainY, testY, model, verbose=True):
    if (verbose): print("Fitting...")
    start = int(round(time()*1000))
    model.fit(
        x=[trainNumX, trainImagesX], y=trainY,
        epochs=EPOCHS, batch_size=8, verbose=False)
    end = int(round(time()*1000))
    train_time = (end-start)/1000
    if (verbose): print("Fitted!")
    return model, train_time

def teste(trainNumX, testNumX, trainImagesX, testImagesX, trainY, testY, train_time, model):
    lescore, pred_time = score(model, testNumX, testImagesX, testY, verbose=False)

    print("After training the model with %s inputs and %s tests, here are the results"%(trainY.shape[0], testY.shape[0]))
    print("-----------")
    print("Time to train (%s inputs, epoch=%s): %s mins - (%s sec)" % (trainY.shape[0], EPOCHS, round(train_time/60, 2), round(train_time, 2)))
    print("Time to predict (%s values) in ms: %s" % (testY.shape[0], pred_time))
    print("Score: %s%%" % (lescore*100))
    print("-----------")

    model.save(SAVE_PATH + "/" + MODEL_NAME)
    return lescore*100

def do(split, which):
    print("--------%s--------" % which)
    total_scores = 0
    total_time = 0
    for i in range(NUM_TIMES_TEST):
        print()
        all_the_data = splite(split, verbose=False)
        model = make(*all_the_data, verbose=False)
        model, train_time = traine(*all_the_data, model)
        the_score = teste(*all_the_data, train_time, model)
        total_scores += the_score
        total_time += train_time
    print("\n")
    return (total_scores/NUM_TIMES_TEST), (total_time/NUM_TIMES_TEST)

In [ ]:
split_q = train_test_split(df, q_images, test_size=0.25, random_state=42)
split_c = train_test_split(df, c_images, test_size=0.25, random_state=42)
splits = [
    [split_q, "Quantum"],
#     [split_c, "Classical"]
]
avgs = []
for er in splits:
    avg = do(*er)
    avgs.append(avg)
    for avg in avgs:
        print()
        print("%s----------" % splits[avgs.index(avg)][1])

    print("Avg Scores:     %s%%" % round(avg[0], 5))
    print("Avg Train Time: %s mins - (%s sec)" % (round(avg[1]/60, 2), round(avg[1], 2)))
print("\n\n///////////////")
print("Using DataVersion %s, Model %s, Img Shape %s, %s slice(s), %s classifications:"
      % (DATA_VERSION, VERSION, IMG_SHAPE, NUM_SLICES, NUM_CLASSES))
for avg in avgs:
    print()
    print("%s----------" % splits[avgs.index(avg)][1])

    print("Avg Scores:     %s%%" % round(avg[0], 5))
    print("Avg Train Time: %s mins - (%s sec)" % (round(avg[1]/60, 2), round(avg[1], 2)))

--------Quantum--------



Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Fitting...


2022-02-11 07:24:46.279711: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fitted!
After training the model with 1152 inputs and 385 tests, here are the results
-----------
Time to train (1152 inputs, epoch=200): 8.16 mins - (489.71 sec)
Time to predict (385 values) in ms: 347
Score: 81.03896103896105%
-----------



Fitting...
Fitted!
After training the model with 1152 inputs and 385 tests, here are the results
-----------
Time to train (1152 inputs, epoch=200): 8.91 mins - (534.84 sec)
Time to predict (385 values) in ms: 403
Score: 81.2987012987013%
-----------



Fitting...
Fitted!
After training the model with 1152 inputs and 385 tests, here are the results
-----------
Time to train (1152 inputs, epoch=200): 8.43 mins - (505.77 sec)
Time to predict (385 values) in ms: 356
Score: 81.81818181818183%
-----------



Fitting...
Fitted!
After training the model with 1152 inputs and 385 tests, here are the results
-----------
Time to train (1152 inputs, epoch=200): 9.37 mins - (562.09 sec)
Time to predict (385 values) in ms: 547
Score: 82.07792207792208%
-------